In [1]:
import os
import ipywidgets as widgets

from src.Settings import RECIPE_DIR
from src.Recipe import Recipe, list_recipes, UNITS, SCALING_FACTORS

UNIT_OPTIONS = [ item for unit in UNITS.values() for item in unit]
DEFAULT_NB_PEOPLE = 4

recipe_list = list_recipes()
recipe_names = list(recipe_list.keys())

edit_drop_down = widgets.HBox(
    [
        widgets.Dropdown(options=recipe_names,
                                description='Load existing',
                                disabled=False),
        widgets.Checkbox(
            value=False,
            description='Edit',
            disabled=False,
            indent=False)
    ]
)

display(edit_drop_down)

            
recipe_name = widgets.Text(value='', placeholder='Recipe Name')
display(recipe_name)
nb_people = widgets.BoundedIntText(value=f"{DEFAULT_NB_PEOPLE}", min=1, max=10, placeholder='quantity')
display(nb_people)

def generate_ingredient_panel():
     return [
        widgets.Label(value="New ingredients: "),
        widgets.HBox([
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip='Add a new ingredient',
                icon='plus',
            ),
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip='Remove an ingredient',
                icon='minus',
            ),
        ]),
    ]
    
ingredients_list = widgets.VBox(generate_ingredient_panel())
display(ingredients_list)

def add_new_ingredient(l=None):
    if isinstance(l, dict):
        widget_list = []
        for k,v in l.items():
            if k in UNITS.keys():
                widget_list.append(widgets.Dropdown(placeholder='Unit', options=UNIT_OPTIONS, value=UNITS[k][0], ensure_option=False, disabled=False))
                widget_list.append(widgets.BoundedFloatText(value=v, min=0, max=1000, placeholder='quantity'))
                break  
        if not widget_list:
            widget_list.append(widgets.Dropdown(placeholder='Unit', options=UNIT_OPTIONS, value='x', ensure_option=False, disabled=False))
            widget_list.append(widgets.BoundedFloatText(value='1', min=0, max=1000, placeholder='quantity'))
                    
        widget_list.append(widgets.Text(value=f"{l.get('name', '')}", placeholder='ingredient name'))
        widget_list.append(widgets.Text(value=f"{l.get('description', '')}", placeholder='description (optional)'))
        ingredients_list.children += (widgets.HBox(widget_list),) 
    else:
        ingredients_list.children += (widgets.HBox(
                [
                    widgets.Dropdown(placeholder='Unit', options=UNIT_OPTIONS, ensure_option=False, disabled=False),
                    widgets.BoundedFloatText(value="1", min=0, max=1000, placeholder='quantity'),
                    widgets.Text(value="", placeholder='ingredient name'),
                    widgets.Text(value="", placeholder='description (optional)'),
                ]
            ),)
        
def remove_last_ingredient(b=None):
    if len(ingredients_list.children) > 2:
        ingredients_list.children = ingredients_list.children[:-1]

ingredients_list.children[1].children[0].on_click(add_new_ingredient) # plus button
ingredients_list.children[1].children[1].on_click(remove_last_ingredient) # minus button

def generate_instruction_panel():
    return [
        widgets.Label(value="New instructions: "),
        widgets.HBox([
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip='Add a new instruction',
                icon='plus',
            ),
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip='Remove an instruction',
                icon='minus',
            ),
        ]),
    ]
    
instructions_list = widgets.VBox(generate_instruction_panel())
display(instructions_list)

def add_new_instruction(l=None):
    if isinstance(l, str):
        instructions_list.children += (widgets.Text(value=l, placeholder='instruction'),)
    else:
        instructions_list.children += (widgets.Text(value="", placeholder='instruction'),)
        
def remove_last_instruction(b=None):
    if len(instructions_list.children) > 2:
        instructions_list.children = instructions_list.children[:-1]

instructions_list.children[1].children[0].on_click(add_new_instruction) # plus button
instructions_list.children[1].children[1].on_click(remove_last_instruction) # minus button

def parse_ingredient_list(ingredients):
    parsed_ingredients = []
    for child in ingredients.children[2:]:
        if isinstance(child, widgets.HBox):
            ingredient = {}
            quantity_key = None
            quantity_value = None
            quantity_scale = None
            for grandchild in child.children:
                if isinstance(grandchild, widgets.Text) and grandchild.placeholder == 'ingredient name':
                    ingredient["name"] = grandchild.value
                if isinstance(grandchild, widgets.Text) and grandchild.placeholder == 'description (optional)' and grandchild.value:
                    ingredient["description"] = grandchild.value
                if isinstance(grandchild, widgets.Dropdown):
                    for k,v in UNITS.items():
                        if grandchild.value in v:
                            quantity_key = k
                            if k in ("weight", "volume") and grandchild.value != v[0]:
                                quantity_scale = SCALING_FACTORS[grandchild.value[0]]
                            else:
                                quantity_scale = 1
                            break
                if isinstance(grandchild, widgets.BoundedFloatText):
                    quantity_value = grandchild.value
            if not ("name" in ingredient.keys() and quantity_key and quantity_value and quantity_scale):
                print(f"Wrong ingredient (idx {len(parsed_ingredients)})")
                print("keys:", ingredient.keys(), "quantity key:", quantity_key, "quantity_value:", quantity_value, "quantity_scale:", quantity_scale)
                return []
            ingredient[quantity_key] = quantity_value * quantity_scale
            parsed_ingredients.append(ingredient)
    
    return parsed_ingredients

def parse_instruction_list(instructions):
    parsed_instructions = []
    for child in instructions.children[2:]:
        if isinstance(child, widgets.Text) and child.placeholder == "instruction":
            parsed_instructions.append(child.value)
                
    return parsed_instructions
                
    

output = widgets.Output()
def verify_recipe(b):
    with output:
        name = recipe_name.value
        if not name:
            print("Missing Recipe name!")
            return None
        if len(ingredients_list.children) <= 2:
            print("Missing ingredients!")
            return None
        ingredients = parse_ingredient_list(ingredients_list)
        if not ingredients:
            print("Missing ingredients!")
            return None
        if len(instructions_list.children) <= 2:
            print("Missing instructions!")
            return None
        instructions = parse_instruction_list(instructions_list)
        if not instructions:
            print("Missing instructions!")
            return None
        
        basename = name.lower().replace(" ", "_").replace("/", "_") + ".yaml"
        if Recipe(name, nb_people.value, ingredients, instructions).save(basename, edit_drop_down.children[1].value):
            print("Recipe saved successfully!")

save_button = widgets.Button(
    disabled=False,
    button_style='',
    tooltip='Save recipe',
    icon='check',
)


display(save_button, output)
save_button.on_click(verify_recipe)

def dropdown_handler(change):
    global nb_people, recipe_name, ingredients_list, instructions_list
    if recipe_name.value:
        nb_people.value = f"{DEFAULT_NB_PEOPLE}"
        recipe_name.value = ""
        ingredients_list.children = (widgets.VBox(generate_ingredient_panel()),)
        instructions_list.children = (widgets.VBox(generate_instruction_panel()),)
    
    print(recipe_list[change["new"]])
    if edit_drop_down.children[1].value: # edit mode
        recipe = Recipe.load(recipe_list[change["new"]]["filename"], recipe_list[change["new"]]["id"])
        recipe_name.value = recipe.name
        nb_people.value = recipe.nb_persons
        for ingredient in recipe.ingredients:
            add_new_ingredient(ingredient)
        for instruction in recipe.instructions:
            add_new_instruction(instruction)
        
edit_drop_down.children[0].observe(dropdown_handler, names='value')

TypeError: 'NoneType' object is not subscriptable